In [0]:
#mount google drive to access data
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'impact/'

#unzip data into the main folder
!unzip -qq ./gdrive/My\ Drive/impact/tiny-imagenet-200.zip

#run the model script the initialize it
%run ./gdrive/My\ Drive/impact/resnet.ipynb

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import math

plt.ion()


data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.48, 0.448, 0.398], [0.276, 0.268, 0.280])
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.48, 0.448, 0.398], [0.276, 0.268, 0.280])
    ]),
}

#read the data
data_dir = 'tiny-imagenet-200'
image_datasets = {x: torchvision.datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=512,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
#script taken from pytorch.org
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [26]:
model= Resnet(BasicBlock,[9,9,9])
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-3,momentum=0.95)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
model = train_model(model, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=90)

Epoch 0/89
----------
train Loss: 4.9924 Acc: 0.0302
val Loss: 6.1934 Acc: 0.0014

Epoch 1/89
----------
train Loss: 4.3758 Acc: 0.0823
val Loss: 7.5991 Acc: 0.0046

Epoch 2/89
----------
train Loss: 3.9644 Acc: 0.1318
val Loss: 7.8841 Acc: 0.0094

Epoch 3/89
----------
train Loss: 3.6908 Acc: 0.1722
val Loss: 7.4868 Acc: 0.0207

Epoch 4/89
----------
train Loss: 3.4732 Acc: 0.2062
val Loss: 8.4530 Acc: 0.0132

Epoch 5/89
----------
train Loss: 3.3391 Acc: 0.2307
val Loss: 9.7152 Acc: 0.0055

Epoch 6/89
----------
train Loss: 3.1041 Acc: 0.2756
val Loss: 9.5383 Acc: 0.0062

Epoch 7/89
----------
train Loss: 3.0533 Acc: 0.2858
val Loss: 9.5177 Acc: 0.0065

Epoch 8/89
----------
train Loss: 3.0280 Acc: 0.2885
val Loss: 9.6869 Acc: 0.0071

Epoch 9/89
----------
train Loss: 3.0066 Acc: 0.2925
val Loss: 9.8107 Acc: 0.0057

Epoch 10/89
----------
train Loss: 2.9835 Acc: 0.2979
val Loss: 9.8304 Acc: 0.0059

Epoch 11/89
----------
train Loss: 2.9614 Acc: 0.3010
val Loss: 9.9523 Acc: 0.0061

Ep

KeyboardInterrupt: ignored

In [27]:
model= Resnet(BasicBlock,[9,9,9])
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-3,momentum=0.95)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
model = train_model(model, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=90)

Epoch 0/89
----------
train Loss: 4.6151 Acc: 0.0616
val Loss: 8.7488 Acc: 0.0058

Epoch 1/89
----------
train Loss: 3.8825 Acc: 0.1451
val Loss: 8.3639 Acc: 0.0010

Epoch 2/89
----------
train Loss: 3.5871 Acc: 0.1883
val Loss: 6.9114 Acc: 0.0105

Epoch 3/89
----------
train Loss: 3.4000 Acc: 0.2205
val Loss: 9.6476 Acc: 0.0018

Epoch 4/89
----------
train Loss: 3.2648 Acc: 0.2437
val Loss: 8.9131 Acc: 0.0027

Epoch 5/89
----------
train Loss: 3.1705 Acc: 0.2641
val Loss: 9.1635 Acc: 0.0021

Epoch 6/89
----------
train Loss: 2.7417 Acc: 0.3479
val Loss: 9.4266 Acc: 0.0082

Epoch 7/89
----------
train Loss: 2.5890 Acc: 0.3773
val Loss: 9.6004 Acc: 0.0076

Epoch 8/89
----------
train Loss: 2.5247 Acc: 0.3900
val Loss: 10.6592 Acc: 0.0059

Epoch 9/89
----------
train Loss: 2.4755 Acc: 0.3970
val Loss: 10.5283 Acc: 0.0062

Epoch 10/89
----------
train Loss: 2.4408 Acc: 0.4051
val Loss: 10.4277 Acc: 0.0048

Epoch 11/89
----------
train Loss: 2.3993 Acc: 0.4126
val Loss: 11.0244 Acc: 0.0072

KeyboardInterrupt: ignored

In [28]:
model= Resnet(BasicBlock,[9,9,9])
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.05, weight_decay=1e-4)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
model = train_model(model, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=90)

Epoch 0/89
----------
train Loss: 7.1927 Acc: 0.0209
val Loss: 9.4059 Acc: 0.0000

Epoch 1/89
----------
train Loss: 4.7237 Acc: 0.0530
val Loss: 6.7972 Acc: 0.0051

Epoch 2/89
----------
train Loss: 4.3682 Acc: 0.0906
val Loss: 7.9418 Acc: 0.0063

Epoch 3/89
----------
train Loss: 4.1006 Acc: 0.1207
val Loss: 8.6290 Acc: 0.0047

Epoch 4/89
----------
train Loss: 3.9453 Acc: 0.1419
val Loss: 7.3336 Acc: 0.0260

Epoch 5/89
----------
train Loss: 3.8387 Acc: 0.1553
val Loss: 9.1781 Acc: 0.0025

Epoch 6/89
----------
train Loss: 3.7760 Acc: 0.1657
val Loss: 7.8528 Acc: 0.0046

Epoch 7/89
----------
train Loss: 3.7363 Acc: 0.1716
val Loss: 8.8918 Acc: 0.0041

Epoch 8/89
----------
train Loss: 3.7187 Acc: 0.1750
val Loss: 7.7090 Acc: 0.0081

Epoch 9/89
----------
train Loss: 3.4438 Acc: 0.2196
val Loss: 8.7631 Acc: 0.0045

Epoch 10/89
----------
train Loss: 3.4007 Acc: 0.2281
val Loss: 7.9398 Acc: 0.0130

Epoch 11/89
----------
train Loss: 3.3909 Acc: 0.2285
val Loss: 8.7832 Acc: 0.0096

Ep

KeyboardInterrupt: ignored